In [233]:
import minsearch
import tiktoken
from elasticsearch import Elasticsearch
from tqdm import tqdm

# Q1. Running Elastic

Run Elastic Search 8.17.6, and get the cluster information. If you run it on localhost, this is how you do it:

curl localhost:9200

What's the version.build_hash value?

42f05b9372a9a4a470db3b52817899b99a76ee73

# Getting the data

Now let's get the FAQ data. You can run this snippet:

In [234]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [235]:
def search(query, index, boost_dict={}, filter_dict={}, num_results=5):

    results = index.search(
        query=query,
        filter_dict=filter_dict,
        boost_dict=boost_dict,
        num_results=num_results
    )

    return results

In [236]:
def create_es_index(index_name, index_settings, es_host='http://localhost:9200'):
    es = Elasticsearch(es_host)

    if es.indices.exists(index=index_name):
        print(f"Index '{index_name}' already exists. Deleting it.")
        es.indices.delete(index=index_name)
        print(f"Creating index '{index_name}' with settings: {index_settings}")
        es.indices.create(index=index_name, body=index_settings)
    return es


def add_documents_to_index(index_name, documents, es_host='http://localhost:9200'):
    es = Elasticsearch(es_host)

    for doc in tqdm(documents, desc=f'Adding documents to {index_name}'):
        es.index(index=index_name, body=doc)

    return es

# Q2. Indexing the data

Index the data in the same way as was shown in the course videos. Make the course field a keyword and the rest should be text.

Don't forget to install the ElasticSearch client for Python:

In [237]:
es_client = Elasticsearch("http://localhost:9200")
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0,
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
        }
    },
}

index_name = "course_questions"

create_es_index(index_name, index_settings, es_host="http://localhost:9200")
add_documents_to_index(index_name, documents, es_host="http://localhost:9200")

Index 'course_questions' already exists. Deleting it.
Creating index 'course_questions' with settings: {'settings': {'number_of_shards': 1, 'number_of_replicas': 0}, 'mappings': {'properties': {'text': {'type': 'text'}, 'section': {'type': 'text'}, 'question': {'type': 'text'}, 'course': {'type': 'keyword'}}}}


Adding documents to course_questions: 100%|██████████| 948/948 [00:14<00:00, 63.91it/s]


<Elasticsearch(['http://localhost:9200'])>

Which function do you use for adding your data to elastic?

- insert
- index
- put
- add

**index**

# Q3. Searching

Now let's search in our index.

We will execute a query "How do execute a command on a Kubernetes pod?".

Use only question and text fields and give question a boost of 4, and use "type": "best_fields".

What's the score for the top ranking result?

- 84.50
- 64.50
- 44.50
- 24.50

Look at the _score field.

**44.50**

In [ ]:
query = 'How do execute a command on a Kubernetes pod?'

search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": [
                {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            ],
        }
    }
}

results = es_client.search(index=index_name, body=search_query)

In [241]:
result_docs = []
for hit in results['hits']['hits']:
    doc = hit['_source']
    doc['score'] = hit['_score']
    result_docs.append(doc)
    result_docs.append(hit['_source'])

result_docs

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp',
  'score': 44.50556},
 {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp',
  'score': 44.

# Q4. Filtering

Now ask a different question: "How do copy a file to a Docker container?".

This time we are only interested in questions from machine-learning-zoomcamp.

Return 3 results. What's the 3rd question returned by the search engine?

- How do I debug a docker container?
- How do I copy files from a different folder into docker container’s working directory?
- How do Lambda container images work?
- How can I annotate a graph?

In [308]:
query = 'How do copy a file to a Docker container?'

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": [
                {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            ],
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

results = es_client.search(index=index_name, body=search_query)

In [309]:
result_docs = []
for hit in results['hits']['hits']:
    doc = hit['_source']
    doc['score'] = hit['_score']
    result_docs.append(doc)
    result_docs.append(hit['_source'])

result_docs = result_docs[:3]  # Limit to 3 results for display
result_docs

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp',
  'score': 73.38676},
 {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp',
  'score': 73.

**How do I copy files from a different folder into docker container’s working directory?**

# Q5. Building a prompt
Now we're ready to build a prompt to send to an LLM.

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (\n\n)

In [310]:
context_template = """
Q: {question}
A: {text}
""".strip()

Now use the context you just created along with the "How do copy a file to a Docker container?" question to construct a prompt using the template below:



In [ ]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT: 
    {context}
    """.strip()

    context = ""
    context_template = """
    Q: {question}
    A: {text}
    """.strip()

    for doc in search_results:
        context += context_template.format(question=doc['question'], text=doc['text']) + "\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [313]:
prompt = build_prompt(query, result_docs)
print(len(prompt))

1412


**1446**

What's the length of the resulting prompt? (use the len function)

- 946
- 1446
- 1946
- 2446

# Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of tokens we send in our prompt and receive in the response.

The OpenAI python package uses tiktoken for tokenization

In [314]:
encoding = tiktoken.encoding_for_model("gpt-4o")

Use the encode function. How many tokens does our prompt have?

- 120
- 220
- 320
- 420

In [315]:
len(encoding.encode(prompt))

316

**320**